In [84]:
%reset -f
import pandas as pd
import requests
import json
from collections import defaultdict

In [85]:
# insert gw of interest
current_gw = 5

In [86]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
fixtures = 'https://fantasy.premierleague.com/api/fixtures/'

In [87]:
# making requests
response = requests.get(url)
fixtures = requests.get(fixtures).json()

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

#print(json.dumps(fixtures, indent=4, sort_keys=True))

The API request was successful.


In [88]:
# mapping teams
teams = response.json()['teams']
team_mapping = {}

for team in teams:
    team_mapping[team['id']] = team['name']

In [89]:
# team difficulty - start of season
team_difficulty = {
    "Arsenal": 6,
    "Aston Villa": 4, 
    "Bournemouth": 2,
    "Brentford": 4,
    "Brighton": 5,
    "Burnley": 1,
    "Chelsea": 4,
    "Crystal Palace": 3,
    "Everton": 2,
    "Fulham": 3, 
    "Liverpool": 6,
    "Luton": 1,
    "Man City": 7,
    "Man United": 5,
    "Newcastle": 5,
    "Nott'm Forest": 3,
    "Sheffield Utd": 1,
    "Spurs": 5,
    "West Ham": 4, 
    "Wolves": 2
}

In [90]:
name_mapping = {
    "Arsenal": "ARS",
    "Aston Villa": "AVL", 
    "Bournemouth": "BOU",
    "Brentford": "BRE",
    "Brighton": "BRI",
    'Burnley': "BUR",
    "Chelsea": "CHE",
    "Crystal Palace": "CRY",
    "Everton": "EVE",
    "Fulham": "FUL", 
    "Liverpool": "LIV",
    "Luton": "LUT",
    "Man City": "MCI",
    "Man Utd": "MUN",
    "Newcastle": "NEW",
    "Nott'm Forest": "NFO",
    "Sheffield Utd": "SHU",
    "Spurs": "TOT",
    "West Ham": "WHU", 
    "Wolves": "WOL"
}

In [91]:
freq = defaultdict(int)
for v in team_difficulty.values():
    freq[v] += 1
    
df = pd.DataFrame.from_dict(freq, orient='index', columns=['Count'])
df.index.name = 'Difficulty'
df = df.sort_index(ascending=False)

df

,Count
Difficulty,
7,1
6,2
5,4
4,4
3,3
2,3
1,3


In [92]:
data = []

for match in fixtures:
    if 'event' not in match or match['event'] is None:
        continue
    elif match['event'] >= current_gw and match['event'] <= (current_gw + 5):
        #print(f"GW {match['event']}: Home: {team_mapping[match['team_h']]} Away:{team_mapping[match['team_a']]}")
        # Extract teams  
        home_team = team_mapping[match['team_h']]
        away_team = team_mapping[match['team_a']]
    
        # append to list as dict
        data.append({
          'event': match['event'],
          'Team': home_team,
          'Team2': away_team
        })

        data.append({
          'event': match['event'],
          'Team': away_team,
          'Team2': home_team
        })

df = pd.DataFrame(data)

# shorten team names
df["Team"] = df["Team"].map(lambda x: name_mapping[x])
df["Team2"] = df["Team2"].map(lambda x: name_mapping[x])

# group by 'event' and 'team1', then concatenate 'team2' values
df['Team2'] = df.groupby(['event', 'Team'])['Team2'].transform(lambda x: ', '.join(x))

# drop duplicates
df.drop_duplicates(subset=['event', 'Team'], inplace=True)

# reset index
df.reset_index(drop=True, inplace=True)

In [93]:
# pivot to wide format
fdr = df.pivot(index='Team', columns='event', values='Team2')

# rename columns 
fdr.columns = [f'GW{col}' for col in fdr.columns]
fdr.fillna('-', inplace = True)

fdr

,GW5,GW6,GW7,GW8,GW9,GW10
Team,,,,,,
ARS,EVE,TOT,BOU,MCI,CHE,SHU
AVL,CRY,CHE,BRI,WOL,WHU,LUT
BOU,CHE,BRI,ARS,EVE,WOL,BUR
BRE,NEW,EVE,NFO,MUN,BUR,CHE
BRI,MUN,BOU,AVL,LIV,MCI,FUL
BUR,NFO,MUN,"NEW, LUT",CHE,BRE,BOU
CHE,BOU,AVL,FUL,BUR,ARS,BRE
CRY,AVL,FUL,MUN,NFO,NEW,TOT
EVE,ARS,BRE,LUT,BOU,LIV,WHU
